In [4]:
import pandas as pd
from faker import Faker
import datetime
import random
from faker.providers import DynamicProvider


class BaseDataGenerator:
    def __init__(self, records):
        self.records = records
        self.fake = Faker('en_IN')

''' *************** Create DataFrame for Table Students ************** '''

class StudentGenerator(BaseDataGenerator):
    def generate_students(self):
        students = []
        for i in range(1, self.records + 1):
            name = self.fake.name()
            enrollment_year = random.randint(2015, 2024)
            graduation_year = random.randint(enrollment_year, enrollment_year + 5)
            students.append({
                "student_id": i,
                "Name": name,
                "Age": random.randint(15, 50),
                "Gender": random.choice(['Male', 'Female', 'Other']),
                "Email": name.lower().replace(' ', '.') + '@fake_domain-2.com',
                "phone": self.fake.phone_number(),
                "enrollment_year": enrollment_year,
                "course_batch": random.choice(['Batch1', 'Batch2', 'Batch3', 'Batch4', 'Batch5', 'Batch6', 'Batch7']),
                "City": self.fake.city(),
                "graduation_year": graduation_year
            })
        return pd.DataFrame(students)

''' *************** Create DataFrame for Table Programming ************** '''

class ProgramGenerator(BaseDataGenerator):
    def __init__(self, records, student_ids):
        super().__init__(records)
        self.student_ids = student_ids

    def generate_program(self):
        skills_provider = DynamicProvider(
            provider_name="skills",
            elements=["Python", "Pandas", "Linux", "SQL", "Data Mining", "Java", "C", "C#", "JavaScript"],
        )
        self.fake.add_provider(skills_provider)

        program_data = []
        for i in range(1,self.records+1):
            problems_solved = random.randint(0, 150)
            assessments_completed = random.randint(0, 30)
            mini_projects = random.randint(0, 30)
            certifications_earned = random.randint(0, 30)
            total_score = problems_solved + assessments_completed + mini_projects + certifications_earned

            program_data.append({
                "programming_id": self.fake.uuid4(),
                "student_id": i,
                "language": self.fake.skills(),
                "problems_solved": problems_solved,
                "assessments_completed": assessments_completed,
                "mini_projects": mini_projects,
                "certifications_earned": certifications_earned,
                "latest_project_score": total_score
            })
        return pd.DataFrame(program_data)

''' *************** Create DataFrame for Table SoftSkills ************** '''

class SkillGenerator(BaseDataGenerator):
    def __init__(self, records, student_ids):
        super().__init__(records)
        self.student_ids = student_ids

    def generate_skills(self):
        skills_data = []
        for i in range(1,self.records+1):
            skills_data.append({
                "soft_skill_id": self.fake.uuid4(),
                "student_id": i,
                "communication": random.randint(0, 100),
                "teamwork": random.randint(0, 100),
                "presentation": random.randint(0, 100),
                "leadership": random.randint(0, 100),
                "critical_thinking": random.randint(0, 100),
                "interpersonal_skills": random.randint(0, 100)
            })
        return pd.DataFrame(skills_data)

''' *************** Create DataFrame for Table Placement ************** '''

class PlacementGenerator(BaseDataGenerator):
    def __init__(self, records, student_ids):
        super().__init__(records)
        self.student_ids = student_ids   
  
    
    def fake_data_generate_placement(self):
        
        company_provider = DynamicProvider(
            provider_name="company",
            elements=["CTS", "TCS", "WIPRO", "INFOSYS", "Amazon", "HCL ", "Capgemini", "IBM", "Accenture"],
        )        
        self.fake.add_provider(company_provider)   
        placement = []

        for i in range(1,self.records+1):
            placement.append({
                    "placement_id" : self.fake.uuid4(),
                    "student_id" : i,
                    "mock_interview_score" : random.randint(0,100),
                    "internships_completed" : random.randint(0,20),
                    "placement_status" : random.choice(['Ready','Not Ready','Placed']),
                    "interview_rounds_cleared" :  random.randint(0,20)
                    })
                          
        return pd.DataFrame(placement)       
        
''' *************** Create DataFrame Only for the students placed ************** '''
                                
    def add_placement_date(self,status):
            if status =='Placed':
                return {
                    "company_name" : self.fake.company(),
                    "placement_package":f"$ {random.randint(20000,50000)}",
                    "placement_date":self.fake.date_time_between(start_date='-5y', end_date='now'),
                    }
                                    
            else: 
                return {
                    "company_name" : None,
                    "placement_package":None,
                    "placement_date":None
                    }

''' *************** Join 2  DataFrames for Table Placement ************** '''

    def generate_placement_dataframe(self):
            placement_df1 = self.fake_data_generate_placement()
            placement_df2 = placement_df1["placement_status"].apply(self.add_placement_date).apply(pd.Series)
            finaldf = pd.concat([placement_df1,placement_df2], axis=1)
            return finaldf


class FakeDataPlacement:
    def __init__(self, records):
        self.records = records

        # Step 1: Generate Students
        self.students_df = StudentGenerator(records).generate_students()
        self.student_ids = self.students_df['student_id'].tolist()

        # Step 2–4: Other data (based on student IDs)
        self.programming_df = ProgramGenerator(records, self.student_ids).generate_program()
        self.skills_df = SkillGenerator(records, self.student_ids).generate_skills()
        self.placement_df = PlacementGenerator(records, self.student_ids).generate_placement_dataframe()

    def get_datasets(self):
        return {
            "students": self.students_df,
            "programming": self.programming_df,
            "softskills": self.skills_df,
            "placements": self.placement_df
        }

    def preview(self):
        return {
            "students": self.students_df.head(),
            "programming": self.programming_df.head(),
            "softskills": self.skills_df.head(),
            "placements": self.placement_df.head()
        }


fx = FakeDataPlacement(50)
data = fx.get_datasets()

# Preview each
for name, df in fx.preview().items():
    print(f"\n{name.upper()} DATA:")
    print(df)
   




{'students':     student_id                Name  Age  Gender  \
0            1    Banjeet Upadhyay   30   Other   
1            2       Daniel Varkey   49  Female   
2            3          Ayaan Tata   15   Other   
3            4        Nisha Thaker   37  Female   
4            5        Tanmayi Gala   37   Other   
5            6         Wishi Nagar   47   Other   
6            7        Advik Sahota   21    Male   
7            8          Viraj Tata   50  Female   
8            9     Onkar Venkatesh   23  Female   
9           10  Pushti Ranganathan   18   Other   
10          11        Mugdha Mahal   22    Male   
11          12      Bina Kuruvilla   21   Other   
12          13           Falak Ram   35  Female   
13          14         Umang Saran   31  Female   
14          15       Farhan Oommen   32  Female   
15          16      Kashish Mannan   30  Female   
16          17    Lopa Ranganathan   41    Male   
17          18        Quincy Uppal   24    Male   
18          19    

In [5]:
from sqlalchemy import create_engine

# Setup
engine = create_engine("mysql+mysqlconnector://{user}:{pw}@{host}/{db}".format(host="127.0.0.1", db="placementeligibility", user="root", pw="NewStart2025"))
#customers_df.to_sql('students', engine, if_exists='append', index=False)

# Export students first
try:
    data["students"].to_sql(name="students", con=engine, if_exists="append", index=False)


    print("Sucessfully exported STUDENTS table to MySQL.")

except SQLAlchemyError as e:
    print("Error occurred:", e)

try:
# Export dependent tables
    for table_name in ["programming", "softskills", "placements"]:
        #data[table_name].to_sql(name=table_name, con=engine, if_exists="replace", index=False)
        data[table_name].to_sql(name=table_name, con=engine, if_exists="append", index=False)
        print(f" Sucessfully exported '{table_name}' table to MySQL.")

except SQLAlchemyError as e:
    print("Error occurred:", e)

Sucessfully exported STUDENTS table to MySQL.
 Sucessfully exported 'programming' table to MySQL.
 Sucessfully exported 'softskills' table to MySQL.
 Sucessfully exported 'placements' table to MySQL.
